In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import scipy.io as sio
import scipy.optimize

import sys
sys.path.append('../../tools/')
import fitting_functions

In [2]:
# Load R interface to do statistics
import rpy2.rinterface
%load_ext rpy2.ipython

C:\Users\jbhasin\AppData\Local\Continuum\anaconda3\lib\site-packages\rpy2\robjects\pandas2ri.py:17: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


# Load short anesthetized fits

In [3]:
def loadBestFits(filename, n):
    fit_file = sio.loadmat(filename)
    lls = fit_file['lls']
    fits = fit_file['fits']
    best_trace_ind = np.argmax(lls[n-1,:])
    best_fits = np.zeros((fits.shape[1], fits[n-1,0].shape[1]))
    for i in range(fits.shape[1]):
        best_fits[i,:] = fits[n-1,i][best_trace_ind,:]
    return best_fits

In [4]:
ms222_traces = ['091311a', '091311b', '091311c', '091311d', '091311e', 
                '091311f', '091411a', '091411d', '091411e', '091411f']

ketamine_traces = ['63011d','70911i', '70911l', '70911m', '82411p', '82411r']

In [5]:
timeconstants_ms222_3 = np.zeros((len(ms222_traces), 3))

for fish_num in range(len(ms222_traces)):
    fish_name = ms222_traces[fish_num]
    fit_file = sio.loadmat('active-comparison/results/MS-222/'+fish_name+'.mat')
    lls_fit = fit_file['lls']
    fits = fit_file['fits']
    best_trace_ind = np.argmax(lls_fit, axis=1)
    timeconstants_ms222_3[fish_num,:] = fits[2,0][best_trace_ind[2],3:]

In [6]:
timeconstants_ketamine_3 = np.zeros((len(ketamine_traces), 3))

for fish_num in range(len(ketamine_traces)):
    fish_name = ketamine_traces[fish_num]
    fit_file = sio.loadmat('active-comparison/results/Ketamine/'+fish_name+'.mat')
    lls_fit = fit_file['lls']
    fits = fit_file['fits']
    best_trace_ind = np.argmax(lls_fit, axis=1)
    timeconstants_ketamine_3[fish_num,:] = fits[2,0][best_trace_ind[2],3:]

### Summary statistics

In [7]:
timeconstants_anesthetized_3 = np.vstack((timeconstants_ms222_3, timeconstants_ketamine_3))

In [22]:
np.median(1/timeconstants_anesthetized_3, axis=0)

array([ 0.1233626 ,  1.93097298, 29.64831403])

In [24]:
for i in range(3):
    print(i+1, np.percentile(1/timeconstants_anesthetized_3[:,i], [25,75], axis=0))

1 [0.10392313 0.13424608]
2 [1.69814827 2.09908591]
3 [25.04221676 38.53749564]


# Load active state fits

In [9]:
# Compare to active state traces
active_traces = [('090711e_0006',), ('090811c_0002',), ('090811d_0002','090811d_0004',),
   ('091111a_0001', '091111a_0003'), ('091111c_0003',), ('091211a_0002', '091211a_0005')]

timeconstants_active_3 = np.zeros((len(active_traces), 3))


for fish_num in range(len(active_traces)):
    fish_name = active_traces[fish_num][0][:-5]
    fit_file = sio.loadmat('../active/fit/results/best/'+fish_name+'.mat')
    lls_fit = fit_file['lls']
    fits = fit_file['fits']
    best_trace_ind = np.argmax(lls_fit, axis=1)
    timeconstants_active_3[fish_num,:] = fits[2,0][best_trace_ind[2],3:]

## Comparison to only MS-222 (10 s) holds

In [12]:
# Compare mean time constant values of 15 s fits to mean active state time constants

avg_timeconstants_15 = 1/np.mean(timeconstants_ms222_3, axis=0)
avg_timeconstants_active = 1/np.mean(timeconstants_active_3, axis=0)
np.abs((avg_timeconstants_active - avg_timeconstants_15)/avg_timeconstants_active)*100

array([23.08056287,  1.32730911, 23.13331244])

In [13]:
# Compare mean time constant values of 15 s fits to mean active state time constants

avg_timeconstants_15 = 1/np.median(timeconstants_ms222_3, axis=0)
avg_timeconstants_active = 1/np.median(timeconstants_active_3, axis=0)
np.abs((avg_timeconstants_active - avg_timeconstants_15)/avg_timeconstants_active)*100

array([30.8780544 , 27.08891622, 18.75444967])

## Comparison to all anesthetized larvae

In [10]:
# Compare mean time constant values of 15 s fits to mean active state time constants

avg_timeconstants_15 = 1/np.mean(timeconstants_anesthetized_3, axis=0)
avg_timeconstants_active = 1/np.mean(timeconstants_active_3, axis=0)
np.abs((avg_timeconstants_active - avg_timeconstants_15)/avg_timeconstants_active)*100

array([20.95049841,  4.0964791 , 28.33090474])

In [11]:
# Compare median time constant values of 15 s fits to mean active state time constants

avg_timeconstants_15 = 1/np.median(timeconstants_anesthetized_3, axis=0)
avg_timeconstants_active = 1/np.median(timeconstants_active_3, axis=0)
np.abs((avg_timeconstants_active - avg_timeconstants_15)/avg_timeconstants_active)*100

array([27.62782077, 23.36268181, 22.59984104])

In [20]:
i = 2
tau_15 = timeconstants_anesthetized_3[:,i]
tau_active = timeconstants_active_3[:,i]

In [21]:
%%R -i tau_15 -i tau_active

wilcox.test(tau_15, tau_active, alternative="two.sided", paired=FALSE, exact=TRUE)

$\tau_1$: W = 72, p-value = 0.08323

$\tau_2$: W = 68, p-value = 0.1545

$\tau_3$: W = 65, p-value = 0.2237